

We create an instance of a SimpleSkyjoEnv environment, call reset() to initialize the game and list the available agents (players):


In [1]:
import random
import numpy as np
from collections import defaultdict

from rlskyjo.environment import simple_skyjo_env 

In [2]:
cfg = {"name": "skyjo_env", "num_players": 2}
env = simple_skyjo_env.env(**cfg)
env.reset()

env.agents, env.agent_selection

def sample_place():
    return np.random.randint(0,11)
        
def sample_draw():
    return np.random.randint(12,13)


In [3]:
print(env.agent_selection)
env.step(sample_draw())
print(env.agent_selection)
env.step(
    sample_place()
)

draw_player_1
12
place_player_1
10


In [4]:
def agent_to_policy(agent):
    
    

IndentationError: expected an indented block (1302384680.py, line 3)

In [7]:
env.reset()
i_episode = 1  
agent_iters = 0
while i_episode <= 1:
    for agent in env.agent_iter(max_iter=600):        
        # get observation (state) for current agent:
        observation, reward, done, info = env.last()
        
        print("training fct:", observation, reward, done, info)
        # perform q-learning with update_Q_value()
        # your code here
        
        env.render(mode='ansi')
        if agent_iters > 600:
            break
        
        # store current state            
        if not done: 
            # choose action using epsilon_greedy_policy()
            # your code here    
            if agent.startswith("draw"):
                action = sample_draw()
            else:
                action = sample_place()
        
            print(f"sampled action {action}")
            env.step(action)
        else: 
            # agent is done
            env.step(None)
            print('done', reward)
            env.render()
            break
    agent_iters = 0
    i_episode  += 1
    env.reset()

observe agent draw_player_0
training fct: {'observation': array([ 0, 16, 16, 16, 16, 16, 16, 10, 16, 16, 16, 16, -4, 10,  2,  0,  1,
        0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  9, 16], dtype=int8), 'action_mask': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1], dtype=int8)} 0 False {}
======= render board: ===== 
======= stats ============ 
next turn: draw by Player 0 
holding card player 0: 16 
discard pile top 9 
======= Player 0 ========== 
 [0	x	x	x]
 [x	x	x	10]
 [x	x	x	x]  
======= Player 1 ========== 
 [x	x	x	-2]
 [x	x	x	x]
 [x	x	x	-2]  

sampled action 12
12
observe agent place_player_0
training fct: {'observation': array([ 0, 16, 16, 16, 16, 16, 16, 10, 16, 16, 16, 16, -4, 10,  2,  0,  1,
        0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  9,  7], dtype=int8), 'action_mask': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], dtype=int8)} 0 False {}
======= render board: ===== 
======= stats ============ 
next turn: place by Player 0 
holding card player 0: 7 
discard pile top 9

AssertionError: 